In [1]:
import os
import re
import json
import joblib
import datetime
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Data Preprocessing

In [3]:
# Dataset path
dataset_path = "/content/drive/MyDrive/AIT/ML/Project/personalized_medical_recommendation/preprocessing/mimic"
processed_path = "/content/drive/MyDrive/AIT/ML/Project/personalized_medical_recommendation/preprocessing/processed"

### 1.1. Diagnoses

In [4]:
# Load MIMIC-III diagnoses table
diagnoses = pd.read_csv(os.path.join(dataset_path, "DIAGNOSES_ICD.csv"))
diagnoses.columns = diagnoses.columns.str.lower()
diagnoses.head()

,row_id,subject_id,hadm_id,seq_num,icd9_code
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


#### Mapping ICD9_CODE to Disease Category

In [5]:
# Path to the downloaded CCS file
ccs_path = "/content/drive/MyDrive/AIT/ML/Project/personalized_medical_recommendation/preprocessing/AppendixASingleDX.txt"

# Dictionary to store ICD-9 to category mapping
icd9_to_category = {}

# Open the file and process line by line
with open(ccs_path, "r") as file:
    current_category = None  # Track current disease category

    for line in file:
        line = line.strip()

        # If the line starts with a number, it's a category ID
        if re.match(r"^\d+\s+[A-Za-z]", line):
            current_category = " ".join(line.split()[1:])  # Extract category name

        # If the line contains only numbers and spaces, it's an ICD-9 code line
        elif re.match(r"^\d+", line) and current_category:
            codes = line.split()  # Split codes by spaces
            for code in codes:
                icd9_to_category[code] = current_category  # Assign category

In [6]:
# Drop rows with nan values
diagnoses.dropna(inplace=True)

# Remove white spaces for icd9_code
diagnoses.icd9_code = diagnoses.icd9_code.str.strip()

# Apply mapping to create a new column for disease categories
# Convert ICD-9 codes to string and map to disease category
diagnoses["disease_category"] = diagnoses["icd9_code"].astype(str).apply(lambda x: icd9_to_category.get(x, "Other"))

In [7]:
# Initialize label encoder
le = LabelEncoder()

# Apply label encoding
diagnoses["disease_category_encoded"] = le.fit_transform(diagnoses["disease_category"])

# Save label mapping for reference (so we can convert back later)
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

In [8]:
# Choose columns
diagnoses = diagnoses[["subject_id", "hadm_id", "icd9_code", "disease_category", "disease_category_encoded"]]
print("Dataset size: ", len(diagnoses))

# Save processed data
diagnoses.to_csv(os.path.join(processed_path, "processed_diagnoses.csv"), index=False)

# Display sample output
diagnoses[["subject_id", "hadm_id", "icd9_code", "disease_category", "disease_category_encoded"]].head()

Dataset size:  651000


,subject_id,hadm_id,icd9_code,disease_category,disease_category_encoded
0,109,172335,40301,Hypertension with complications and secondary ...,114
1,109,172335,486,Pneumonia (except that caused by tuberculosis ...,213
2,109,172335,58281,Nephritis; nephrosis; renal sclerosis,148
3,109,172335,5855,Chronic kidney disease,55
4,109,172335,4254,Peri-; endo-; and myocarditis; cardiomyopathy ...,207


In [9]:
# Convert NumPy int64 keys/values to Python int/str
label_mapping_json = {str(k): int(v) for k, v in label_mapping.items()}

# Save the mapping as a JSON file
with open(os.path.join(processed_path, "disease_category_mapping.json"), "w") as f:
    json.dump(label_mapping_json, f)

# To load the mapping back later:
# with open("mimic/disease_category_mapping.json", "r") as f:
#     label_mapping = json.load(f)

print("Disease category mapping saved!")

Disease category mapping saved!


In [10]:
del diagnoses, label_mapping_json

### 1.2. Prescriptions

In [11]:
# Load MIMIC-III prescriptions table
prescriptions = pd.read_csv(os.path.join(dataset_path, "PRESCRIPTIONS.csv"))
prescriptions.columns = prescriptions.columns.str.lower()
prescriptions.head()

<ipython-input-11-e370ea876d96>:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  prescriptions = pd.read_csv(os.path.join(dataset_path, "PRESCRIPTIONS.csv"))


,row_id,subject_id,hadm_id,icustay_id,startdate,enddate,drug_type,drug,drug_name_poe,drug_name_generic,formulary_drug_cd,gsn,ndc,prod_strength,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,route
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,469061711.0,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,56017275.0,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,54829725.0,20mg Tablet,20,mg,1,TAB,PO


In [12]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Load PubMedBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")
model = AutoModel.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")

def get_pubmedbert_embedding(drug_name):
    """Generate embedding using PubMedBERT for a given drug name."""
    inputs = tokenizer(drug_name, return_tensors="pt", truncation=True, padding=True, max_length=50)
    with torch.no_grad():
        output = model(**inputs)
    return output.last_hidden_state.mean(dim=1).squeeze().numpy()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [13]:
# Extract unique medication names
medications = prescriptions["drug"].dropna().unique()

# Generate PubMedBERT embeddings for each medication
medication_embeddings = {drug: get_pubmedbert_embedding(drug) for drug in medications}

# Apply PCA before converting to DataFrame
embedding_matrix = np.stack(list(medication_embeddings.values()))
drug_list = list(medication_embeddings.keys())

pca = PCA(n_components=128)
embedding_matrix_reduced = pca.fit_transform(embedding_matrix)

# Replace values in the same dictionary with reduced embeddings
medication_embeddings = {
    drug: embedding_matrix_reduced[i] for i, drug in enumerate(drug_list)
}

# Convert embeddings into a DataFrame (same as before)
medication_df = pd.DataFrame.from_dict(medication_embeddings, orient="index")
medication_df.reset_index(inplace=True)
medication_df.columns = ["drug"] + [f"dim_{i}" for i in range(medication_df.shape[1] - 1)]

# Save medication embeddings
medication_df.to_csv(os.path.join(processed_path, "medication_embeddings.csv"), index=False)

# Save PCA model
joblib.dump(pca, os.path.join(processed_path, "pca_pubmedbert_768to128.pkl"))

print("Generated PubMedBERT embeddings (with PCA) for medications!")

Generated PubMedBERT embeddings (with PCA) for medications!


In [14]:
# Choose columns
prescriptions = prescriptions[['subject_id', 'hadm_id', 'drug', 'startdate', 'enddate']]

prescriptions['startdate'] = pd.to_datetime(prescriptions['startdate'])
prescriptions['enddate'] = pd.to_datetime(prescriptions['enddate'])

# Merge embeddings into the prescriptions dataset
prescriptions = prescriptions.merge(medication_df, on="drug", how="left")
print("Dataset size: ", len(prescriptions))

# Save the updated prescriptions data with embeddings
prescriptions.to_csv(os.path.join(processed_path, "processed_prescriptions.csv"), index=False)

# Display sample results
prescriptions[["subject_id", "hadm_id", "drug", "startdate", "enddate"] + [f"dim_{i}" for i in range(5)]].head()

Dataset size:  4156450


,subject_id,hadm_id,drug,startdate,enddate,dim_0,dim_1,dim_2,dim_3,dim_4
0,6,107064,Tacrolimus,2175-06-11,2175-06-12,3.596636,-6.513249,-1.840859,-0.239163,2.239905
1,6,107064,Warfarin,2175-06-11,2175-06-12,3.201997,-6.879823,-0.593458,-0.111228,2.152487
2,6,107064,Heparin Sodium,2175-06-11,2175-06-12,0.416288,-2.453712,-2.032402,-2.062383,0.602265
3,6,107064,D5W,2175-06-11,2175-06-12,-0.992017,-1.524479,-0.571861,-2.774142,-0.930367
4,6,107064,Furosemide,2175-06-11,2175-06-12,3.938535,-4.577149,-2.282591,-1.172301,0.494638


In [15]:
prescriptions.head()

,subject_id,hadm_id,drug,startdate,enddate,dim_0,dim_1,dim_2,dim_3,dim_4,...,dim_118,dim_119,dim_120,dim_121,dim_122,dim_123,dim_124,dim_125,dim_126,dim_127
0,6,107064,Tacrolimus,2175-06-11,2175-06-12,3.596636,-6.513249,-1.840859,-0.239163,2.239905,...,-0.123296,-0.167815,-0.036730,-0.104513,-0.007000,-0.271453,-0.221180,0.128914,0.308851,-0.099569
1,6,107064,Warfarin,2175-06-11,2175-06-12,3.201997,-6.879823,-0.593458,-0.111228,2.152487,...,-0.158901,-0.095175,-0.325314,-0.101868,-0.178969,0.125522,-0.074548,0.299590,0.242968,-0.163216
2,6,107064,Heparin Sodium,2175-06-11,2175-06-12,0.416288,-2.453712,-2.032402,-2.062383,0.602265,...,-0.178576,0.134767,0.293970,-0.096392,0.235018,-0.147567,0.195223,-0.239148,0.145286,-0.053593
3,6,107064,D5W,2175-06-11,2175-06-12,-0.992017,-1.524479,-0.571861,-2.774142,-0.930367,...,0.274008,-0.390862,-0.274539,-0.389282,0.375123,-0.407941,0.389927,0.102683,-0.174294,0.955514
4,6,107064,Furosemide,2175-06-11,2175-06-12,3.938535,-4.577149,-2.282591,-1.172301,0.494638,...,-0.176882,-0.459282,-0.007256,-0.123898,0.356509,-0.140138,0.001122,-0.310104,-0.089182,0.329590


In [16]:
del prescriptions, medication_df

### 1.3. Admissions

In [17]:
# Load MIMIC-III ADMISSIONS table
admissions = pd.read_csv(os.path.join(dataset_path, "ADMISSIONS.csv"))
admissions.columns = admissions.columns.str.lower()
admissions.head()

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [18]:
# Select the first ethnicity recorded for each patient
patient_ethnicity = admissions.groupby("subject_id")["ethnicity"].first().reset_index()

# Encode ethnicity as numerical labels
patient_ethnicity["ethnicity"], ethnicity_mapping = pd.factorize(patient_ethnicity["ethnicity"])

# Display sample mapping
print("Ethnicity Mapping:", dict(enumerate(ethnicity_mapping)))
patient_ethnicity.head()

Ethnicity Mapping: {0: 'ASIAN', 1: 'WHITE', 2: 'UNKNOWN/NOT SPECIFIED', 3: 'BLACK/AFRICAN AMERICAN', 4: 'PATIENT DECLINED TO ANSWER', 5: 'OTHER', 6: 'HISPANIC OR LATINO', 7: 'HISPANIC/LATINO - GUATEMALAN', 8: 'HISPANIC/LATINO - PUERTO RICAN', 9: 'ASIAN - ASIAN INDIAN', 10: 'ASIAN - VIETNAMESE', 11: 'MULTI RACE ETHNICITY', 12: 'HISPANIC/LATINO - DOMINICAN', 13: 'AMERICAN INDIAN/ALASKA NATIVE', 14: 'WHITE - RUSSIAN', 15: 'HISPANIC/LATINO - SALVADORAN', 16: 'ASIAN - CHINESE', 17: 'UNABLE TO OBTAIN', 18: 'BLACK/CAPE VERDEAN', 19: 'WHITE - OTHER EUROPEAN', 20: 'PORTUGUESE', 21: 'BLACK/HAITIAN', 22: 'ASIAN - CAMBODIAN', 23: 'SOUTH AMERICAN', 24: 'ASIAN - FILIPINO', 25: 'CARIBBEAN ISLAND', 26: 'WHITE - EASTERN EUROPEAN', 27: 'HISPANIC/LATINO - COLOMBIAN', 28: 'WHITE - BRAZILIAN', 29: 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER', 30: 'HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)', 31: 'ASIAN - JAPANESE', 32: 'ASIAN - THAI', 33: 'BLACK/AFRICAN', 34: 'HISPANIC/LATINO - HONDURAN', 35: 'MIDDLE EASTER

,subject_id,ethnicity
0,2,0
1,3,1
2,4,1
3,5,0
4,6,1


In [19]:
# Convert datetime columns
admissions["admittime"] = pd.to_datetime(admissions["admittime"])
admissions["dischtime"] = pd.to_datetime(admissions["dischtime"])

# Calculate length of hospital stay
admissions["length_of_stay"] = (admissions["dischtime"] - admissions["admittime"]).dt.days

admissions = admissions[['subject_id', 'hadm_id', 'admittime', 'dischtime', 'length_of_stay', 'admission_type']]
print("Dataset size: ", len(admissions))

# Save the admission csv
admissions.to_csv(os.path.join(processed_path, "processed_admissions.csv"), index=False)

# Display updated admissions data
admissions.head()

Dataset size:  58976


,subject_id,hadm_id,admittime,dischtime,length_of_stay,admission_type
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,1,EMERGENCY
1,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,5,ELECTIVE
2,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,6,EMERGENCY
3,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,2,EMERGENCY
4,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,3,EMERGENCY


### 1.4. Lab Events

In [20]:
# Load LABEVENTS table
labevents = pd.read_csv(os.path.join(dataset_path, "LABEVENTS.csv"))
labevents.columns = labevents.columns.str.lower()
labevents.head()

,row_id,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,flag
0,281,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN
1,282,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN
2,283,3,NaN,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,NaN
3,284,3,NaN,50804,2101-10-12 18:17:00,22,22.00,mEq/L,NaN
4,285,3,NaN,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal


#### Fill nan in hadm_id with the patient's first hospital admission

In [21]:
# Get first hospital admission for each patient
first_admission = admissions.groupby("subject_id")["hadm_id"].first().reset_index()

# Merge first admission ID into lab events
labevents = labevents.merge(first_admission, on="subject_id", how="left", suffixes=("", "_first"))

# Fill NaN hadm_id with first known hospital admission
labevents["hadm_id"] = labevents["hadm_id"].fillna(labevents["hadm_id_first"])

# Drop temporary column
labevents = labevents.drop(columns=["hadm_id_first"])

In [22]:
# Convert datetime column
labevents["charttime"] = pd.to_datetime(labevents["charttime"])

# Drop missing values in lab results
labevents = labevents.dropna(subset=["valuenum"])

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Normalize lab values (Min-Max Scaling)
labevents["valuenum_normalized"] = scaler.fit_transform(labevents[["valuenum"]])

labevents = labevents[['subject_id', 'hadm_id', 'itemid', 'charttime', 'valuenum_normalized']]
print("Dataset size: ", len(labevents))

# Save processed lab events dataset
labevents.to_csv(os.path.join(processed_path, "processed_labevents.csv"), index=False)

# Display sample normalized lab results
labevents.head()

Dataset size:  24932835


,subject_id,hadm_id,itemid,charttime,valuenum_normalized
0,3,145834.0,50820,2101-10-12 16:07:00,0.000030
2,3,145834.0,50802,2101-10-12 18:17:00,0.000029
3,3,145834.0,50804,2101-10-12 18:17:00,0.000031
4,3,145834.0,50808,2101-10-12 18:17:00,0.000029
6,3,145834.0,50813,2101-10-12 18:17:00,0.000029


In [23]:
# Save the scaler
joblib.dump(scaler, os.path.join(processed_path, "scaler_valuenum_labevents.pkl"))
print("MinMaxScaler saved successfully!")

MinMaxScaler saved successfully!


In [24]:
del labevents

### 1.5. Procedures

In [25]:
# Load PROCEDURES_ICD table
procedures = pd.read_csv(os.path.join(dataset_path, "PROCEDURES_ICD.csv"))
procedures.columns = procedures.columns.str.lower()
procedures.head()

,row_id,subject_id,hadm_id,seq_num,icd9_code
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331


In [26]:
# Path to the CCS procedures file
ccs_path = "/content/drive/MyDrive/AIT/ML/Project/personalized_medical_recommendation/preprocessing/AppendixBSinglePR.txt"

# Dictionary to store ICD-9 to category mapping
icd9_proc_to_category = {}

# Open the file and process line by line
with open(ccs_path, "r") as file:
    current_category = None  # Track current disease category

    for line in file:
        line = line.strip()

        # If the line starts with a number, it's a category ID
        if re.match(r"^\d+\s+[A-Za-z]", line):
            current_category = " ".join(line.split()[1:])  # Extract category name

        # If the line contains only numbers and spaces, it's an ICD-9 code line
        elif re.match(r"^\d+", line) and current_category:
            codes = line.split()  # Split codes by spaces
            for code in codes:
                icd9_proc_to_category[code] = current_category  # Assign category

In [27]:
# Convert ICD-9 procedure codes to string and apply mapping
procedures["procedure_category"] = procedures["icd9_code"].astype(str).apply(lambda x: icd9_proc_to_category.get(x, "Other"))

In [28]:
# Initialize label encoder
le = LabelEncoder()

# Apply label encoding
procedures["procedure_category_encoded"] = le.fit_transform(procedures["procedure_category"])

# Save label mapping for reference (so we can convert back later)
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

In [29]:
# Choose columns
procedures = procedures[['subject_id', 'hadm_id', 'icd9_code', 'procedure_category', 'procedure_category_encoded']]
print("Dataset size: ", len(procedures))

# Save processed procedures dataset
procedures.to_csv(os.path.join(processed_path, "processed_procedures.csv"), index=False)

# Display sample output
procedures.head()

Dataset size:  240095


,subject_id,hadm_id,icd9_code,procedure_category,procedure_category_encoded
0,62641,154460,3404,Incision of pleura; thoracentesis; chest drainage,84
1,2592,130856,9671,Respiratory intubation and mechanical ventilation,191
2,2592,130856,3893,Other vascular catheterization; not heart,174
3,55357,119355,9672,Respiratory intubation and mechanical ventilation,191
4,55357,119355,331,Other OR Rx procedures on respiratory system a...,119


In [30]:
# Convert NumPy int64 keys/values to Python int/str
label_mapping_json = {str(k): int(v) for k, v in label_mapping.items()}

# Save the mapping as a JSON file
with open(os.path.join(processed_path, "procedure_category_mapping.json"), "w") as f:
    json.dump(label_mapping_json, f)

# To load the mapping back later:
# with open("mimic/procedure_category_mapping.json", "r") as f:
#     label_mapping = json.load(f)

print("Procedure category mapping saved!")

Procedure category mapping saved!


In [31]:
del procedures

### 1.6. Patients

In [32]:
# Load MIMIC-III PATIENTS table
patients = pd.read_csv(os.path.join(dataset_path, "PATIENTS.csv"))
patients.columns = patients.columns.str.lower()
patients.head()

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0


In [33]:
# Get first admission year for each patient
first_admission = admissions.groupby("subject_id")["admittime"].min().reset_index()
first_admission["admission_year"] = first_admission["admittime"].dt.year

# Merge admission year with patient data
patients = patients.merge(first_admission[["subject_id", "admission_year"]], on="subject_id", how="left")

# Convert DOB to Age
patients["dob"] = pd.to_datetime(patients["dob"])  # Convert to datetime
patients["age"] = patients["admission_year"] - patients["dob"].dt.year

In [34]:
# Convert gender to binary (M=1, F=0)
patients["gender"] = patients["gender"].apply(lambda x: 1 if x == "M" else 0)

# Merge Ethnicity from Admissions
patients = patients.merge(patient_ethnicity, on="subject_id", how="left")

patients = patients[['subject_id', 'age', 'gender', 'ethnicity']]
print("Dataset size: ", len(patients))

# Save processed patients dataset
patients.to_csv(os.path.join(processed_path, "processed_patients.csv"))

# Display sample output
patients.head()

Dataset size:  46520


,subject_id,age,gender,ethnicity
0,249,74,0,1
1,250,24,0,3
2,251,20,1,2
3,252,55,1,1
4,253,85,0,1


In [35]:
del patients